In [123]:
import os

# Print current working directory
print("Current working dir:", os.getcwd())

# Check if the dataset path exists
print("Dataset path exists:", os.path.exists("processed_data/transformer_dataset"))

Current working dir: /Users/PetersMacBook/Documents/GitHub/ST456 - Deep Learning/project/2025-projects-shallowlearning/code
Dataset path exists: True


In [124]:
NUM_ENTITIES: int = 23
COORDS: list[str] = ["x", "y", "s", "a", "dir_sin", "dir_cos"]

POSITIONAL_FEATURES = [f"{c}_{i}"
                  for i in range(1, NUM_ENTITIES + 1)
                  for c in COORDS]

print(POSITIONAL_FEATURES)

['x_1', 'y_1', 's_1', 'a_1', 'dir_sin_1', 'dir_cos_1', 'x_2', 'y_2', 's_2', 'a_2', 'dir_sin_2', 'dir_cos_2', 'x_3', 'y_3', 's_3', 'a_3', 'dir_sin_3', 'dir_cos_3', 'x_4', 'y_4', 's_4', 'a_4', 'dir_sin_4', 'dir_cos_4', 'x_5', 'y_5', 's_5', 'a_5', 'dir_sin_5', 'dir_cos_5', 'x_6', 'y_6', 's_6', 'a_6', 'dir_sin_6', 'dir_cos_6', 'x_7', 'y_7', 's_7', 'a_7', 'dir_sin_7', 'dir_cos_7', 'x_8', 'y_8', 's_8', 'a_8', 'dir_sin_8', 'dir_cos_8', 'x_9', 'y_9', 's_9', 'a_9', 'dir_sin_9', 'dir_cos_9', 'x_10', 'y_10', 's_10', 'a_10', 'dir_sin_10', 'dir_cos_10', 'x_11', 'y_11', 's_11', 'a_11', 'dir_sin_11', 'dir_cos_11', 'x_12', 'y_12', 's_12', 'a_12', 'dir_sin_12', 'dir_cos_12', 'x_13', 'y_13', 's_13', 'a_13', 'dir_sin_13', 'dir_cos_13', 'x_14', 'y_14', 's_14', 'a_14', 'dir_sin_14', 'dir_cos_14', 'x_15', 'y_15', 's_15', 'a_15', 'dir_sin_15', 'dir_cos_15', 'x_16', 'y_16', 's_16', 'a_16', 'dir_sin_16', 'dir_cos_16', 'x_17', 'y_17', 's_17', 'a_17', 'dir_sin_17', 'dir_cos_17', 'x_18', 'y_18', 's_18', 'a_18', '

In [125]:
from pathlib import Path
import tensorflow as tf
import numpy as np
import statistics as st

In [126]:
dataset_path = "processed_data/transformer_dataset"

# Load dataset without any transformations
raw_ds = tf.data.Dataset.load(dataset_path)

# Print dataset structure
print("Dataset element specification:", raw_ds.element_spec)

# Examine first 3 examples
print("\nFirst 3 examples:")
for i, example in enumerate(raw_ds.take(3)):
    # Each example contains 3 components:
    meta_tensor = example[0]  # Metadata (gameId, playId, split_id, firstFrameId)
    x_tensor = example[1]     # Input sequence (padded frames)
    y_tensor = example[2]     # Target vector
    
    print(f"\nExample {i+1}:")
    print("Metadata tensor:", meta_tensor)
    print(f"Metadata values: {meta_tensor.numpy()}")
    print(f"Input shape: {x_tensor.shape} | dtype: {x_tensor.dtype}")
    print(f"Target shape: {y_tensor.shape} | dtype: {y_tensor.dtype}")
    
    # First 5 elements of first frame's features
    print("First frame features (first 5 values):", x_tensor[0, :5].numpy())
    print("Target values (first 5):", y_tensor[:5].numpy())
    

Dataset element specification: (TensorSpec(shape=(4,), dtype=tf.int32, name=None), TensorSpec(shape=(100, 138), dtype=tf.float32, name=None), TensorSpec(shape=(138,), dtype=tf.float32, name=None))

First 3 examples:

Example 1:
Metadata tensor: tf.Tensor([2022090800       1102          0          1], shape=(4,), dtype=int32)
Metadata values: [2022090800       1102          0          1]
Input shape: (100, 138) | dtype: <dtype: 'float32'>
Target shape: (138,) | dtype: <dtype: 'float32'>
First frame features (first 5 values): [0. 0. 0. 0. 0.]
Target values (first 5): [0.6975     0.7036667  0.67441666 0.69916666 0.68475   ]

Example 2:
Metadata tensor: tf.Tensor([2022090800       1102          0          1], shape=(4,), dtype=int32)
Metadata values: [2022090800       1102          0          1]
Input shape: (100, 138) | dtype: <dtype: 'float32'>
Target shape: (138,) | dtype: <dtype: 'float32'>
First frame features (first 5 values): [0. 0. 0. 0. 0.]
Target values (first 5): [0.69775    0.7

In [127]:
# Function to filter based on split_id
def filter_split(split_num):
    def _filter(meta, x, y):
        return tf.equal(meta[2], split_num)
    return _filter


In [128]:
# Examine first 3 examples
print("\nFirst 3 examples:")
for i, example in enumerate(train_ds.take(3)):
    # Each example contains 3 components:
    meta_tensor = example[0]  # Metadata (gameId, playId, split_id, firstFrameId)
    x_tensor = example[1]     # Input sequence (padded frames)
    y_tensor = example[2]     # Target vector
    
    print(f"\nExample {i+1}:")
    print("Metadata tensor:", meta_tensor)
    print(f"Metadata values: {meta_tensor.numpy()}")
    print(f"Input shape: {x_tensor.shape} | dtype: {x_tensor.dtype}")
    print(f"Target shape: {y_tensor.shape} | dtype: {y_tensor.dtype}")
    
    # First 5 elements of first frame's features
    print("First frame features (first 5 values):", x_tensor[0, :-5].numpy())
    print("Target values (first 5):", y_tensor[:5].numpy())


First 3 examples:

Example 1:
Metadata tensor: tf.Tensor(
[[2022091106       2191          0          1]
 [2022090800       2360          0         61]
 [2022091112        552          0         10]
 [2022091110       3357          0          1]
 [2022090800       1102          0        127]
 [2022090800       3431          0          1]
 [2022091104        217          0          1]
 [2022091100        741          0          1]
 [2022091108       1060          0          1]
 [2022091108       3227          0          1]
 [2022091102       2532          0          1]
 [2022091103       3568          0          1]
 [2022091112        552          0         51]
 [2022091108       1378          0          1]
 [2022091102       2532          0          1]
 [2022090800       1102          0         35]
 [2022091102       2532          0         15]
 [2022090800       2307          0          1]
 [2022091109       3357          0          1]
 [2022091103       3435          0          1]
 

In [129]:
print("Dataset cardinality:",
      tf.data.experimental.cardinality(raw_ds).numpy())   # should now print a number


Dataset cardinality: 304996


In [146]:
def drop_meta(meta, x, y):
    # x: (T, F), y: (138,)
    # Keep only last frame from x (time t) and pass to loss as part of y
    x_t = x[-1]  # shape (138,) — same as y
    return x, tf.concat([x_t, y], axis=-1)

train_ds = (raw_ds
            .filter(filter_split(0))
            .map(drop_meta, num_parallel_calls=tf.data.AUTOTUNE)
            .shuffle(4096)
            .batch(64)
            .prefetch(tf.data.AUTOTUNE))

val_ds   = (raw_ds
            .filter(filter_split(1))
            .map(drop_meta, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(64)
            .prefetch(tf.data.AUTOTUNE))

test_ds  = (raw_ds
            .filter(filter_split(2))
            .map(drop_meta, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(64)
            .prefetch(tf.data.AUTOTUNE))

    

In [150]:
# Take one batch from the dataset

for x_batch, y_true_flat in train_ds.take(1):
    print("x_batch shape:", x_batch.shape)         # (64, 100, 138)
    print("y_true_flat shape:", y_true_flat.shape) # (64, 276)

for x_batch, y_true_flat in val_ds.take(1):
    print("x_batch shape:", x_batch.shape)         # (64, 100, 138)
    print("y_true_flat shape:", y_true_flat.shape) # (64, 276)

for x_batch, y_true_flat in test_ds.take(1):
    print("x_batch shape:", x_batch.shape)         # (64, 100, 138)
    print("y_true_flat shape:", y_true_flat.shape) # (64, 276)

x_batch shape: (64, 100, 138)
y_true_flat shape: (64, 276)
x_batch shape: (64, 100, 138)
y_true_flat shape: (64, 276)
x_batch shape: (64, 100, 138)
y_true_flat shape: (64, 276)


In [151]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

NUM_FEATS = 138          # x,y for 23 entities
MAX_LEN  = 100          # same value you used in dataset builder
D_MODEL  = 128          # transformer hidden size
N_HEADS  = 4
N_LAYERS = 4
D_FF     = 512
DROPOUT  = 0.1

In [152]:
# ╔═══════════════════╗
# ║ 2. Positional enc ║  (learnable 1‑D embedding)
# ╚═══════════════════╝
class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.pos_emb = self.add_weight(
            name="pos_emb",
            shape=(max_len, d_model),
            initializer="uniform",
            trainable=True,
        )

    def call(self, x):
        return x + self.pos_emb


In [153]:
# ╔═══════════════════════════╗
# ║ 3. Padding‑mask function  ║
# ╚═══════════════════════════╝
class PaddingMask(layers.Layer):
    def call(self, x):
        # x:  (B, T, F) — zero‐padded on the left
        pad = tf.reduce_all(tf.equal(x, 0.0), axis=-1)      # → (B, T)
        # reshape to (B, 1, 1, T) for MultiHeadAttention
        return pad[:, tf.newaxis, tf.newaxis, :]



In [154]:
# ╔════════════════════════╗
# ║ 4. Transformer encoder ║
# ╚════════════════════════╝
def transformer_block(d_model, n_heads, d_ff, dropout):
    inputs   = layers.Input(shape=(None, d_model))
    padding  = layers.Input(shape=(1,1,None), dtype=tf.bool)  # mask

    x = layers.MultiHeadAttention(
        num_heads=n_heads, key_dim=d_model//n_heads, dropout=dropout
    )(inputs, inputs, attention_mask=padding)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(inputs + x)

    y = layers.Dense(d_ff, activation="relu")(x)
    y = layers.Dense(d_model)(y)
    y = layers.Dropout(dropout)(y)
    y = layers.LayerNormalization(epsilon=1e-6)(x + y)

    return keras.Model([inputs, padding], y)


In [155]:
# ╔════════════════════════════════╗
# ║ 5. End‑to‑end prediction model ║
# ╚════════════════════════════════╝
def build_model(
    num_feats=NUM_FEATS,
    max_len=MAX_LEN,
    d_model=D_MODEL,
    n_heads=N_HEADS,
    n_layers=N_LAYERS,
    d_ff=D_FF,
    dropout=DROPOUT,
):
    seq_in  = layers.Input(shape=(max_len, num_feats), name="sequence")   # (B,T,F)

    # Linear projection to d_model
    x = layers.Dense(d_model)(seq_in)

    # Add learnable positional encodings
    x = PositionalEncoding(max_len, d_model)(x)

    # Build padding mask once
    pad_mask = PaddingMask()(seq_in)

    # Stack encoder layers
    for _ in range(n_layers):
        x = transformer_block(d_model, n_heads, d_ff, dropout)([x, pad_mask])

    # We need the hidden state that corresponds to *frame t* (the last row)
    # – that is always index -1 thanks to left padding.
    h_t = layers.Lambda(lambda t: t[:, -1])(x)          # (B, D)

    # Predict only (acceleration, dir_sin, dir_cos) x 22 players + ball = 69 features
    out = layers.Dense(69, name="pred_accel_dir")(h_t)

    return keras.Model(seq_in, out, name="NFL_Frame_Predictor")

model = build_model()
model.summary()


Model: "NFL_Frame_Predictor"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sequence            │ (None, 100, 138)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_63 (Dense)    │ (None, 100, 128)  │     17,792 │ sequence[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encodin… │ (None, 100, 128)  │     12,800 │ dense_63[0][0]    │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ padding_mask_7      │ (None, 1, 1, 100) │          0 │ sequence[0][0]    │
│ (PaddingMask)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_28       │ (None, 100, 128)  │    198,272 │ positional_encod… │
│ (Functional)        │                   │            │ padding_mask_7[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_29       │ (None, 100, 128)  │    198,272 │ functional_28[0]… │
│ (Functional)        │                   │            │ padding_mask_7[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_30       │ (None, 100, 128)  │    198,272 │ functional_29[0]… │
│ (Functional)        │                   │            │ padding_mask_7[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_31       │ (None, 100, 128)  │    198,272 │ functional_30[0]… │
│ (Functional)        │                   │            │ padding_mask_7[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_7 (Lambda)   │ (None, 128)       │          0 │ functional_31[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pred_accel_dir      │ (None, 69)        │      8,901 │ lambda_7[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 832,581 (3.18 MB)

 Trainable params: 832,581 (3.18 MB)

 Non-trainable params: 0 (0.00 B)

In [156]:
# ╔═════════════════════════╗
# ║ 6. Define loss function ║
# ╚═════════════════════════╝

def physics_informed_loss_accel(y_true, y_pred, x_t, s_true, delta_t=0.1):
    """
    Physics-informed loss for NFL tracking.
    
    Inputs:
      - y_true: (batch, 23, 2) true position at t+1
      - y_pred: (batch, 23, 3) predicted (acceleration, dir_sin, dir_cos)
      - x_t:    (batch, 23, 2) starting position at t
      - s_true: (batch, 23) speed at t (unscaled)
    """

    # Unpack predictions
    a_pred = y_pred[:, :, 0]         # acceleration magnitude
    dir_sin_pred = y_pred[:, :, 1]
    dir_cos_pred = y_pred[:, :, 2]

    # Reconstruct initial velocity
    v_x_init = s_true * dir_cos_pred
    v_y_init = s_true * dir_sin_pred

    # Acceleration components
    a_x = a_pred * dir_cos_pred
    a_y = a_pred * dir_sin_pred

    # Update velocity
    v_x = v_x_init + a_x * delta_t
    v_y = v_y_init + a_y * delta_t

    # Predict new position
    x_t1_pred = x_t[:, :, 0] + v_x * delta_t
    y_t1_pred = x_t[:, :, 1] + v_y * delta_t

    position_pred = tf.stack([x_t1_pred, y_t1_pred], axis=-1)

    # Loss: MSE between predicted position and ground-truth
    loss = tf.reduce_mean(tf.square(y_true - position_pred))

    return loss

In [ ]:
# class PhysicsLossWrapper(tf.keras.losses.Loss):
#     def __init__(self, delta_t=0.1):
#         super().__init__()
#         self.delta_t = delta_t

#     def call(self, y_true, y_pred):
#         """
#         y_true: (batch, 138)
#         y_pred: (batch, 69)
        
#         y_true is ordered as:
#         [x1, y1, s1, a1, dir_sin1, dir_cos1, ..., x23, y23, s23, a23, dir_sin23, dir_cos23]
#         """
#         x_t, y_target = y_true

#         # 1. Reshape y_true into (batch, 23 entities, 6 features)
#         y_target_reshaped = tf.reshape(y_target, (-1, 23, 6))  # (batch, 23, 6)

#         x_t_reshaped = tf.reshape(x_t[:, -1, :], (-1, 23, 6))  # last timestep
     
#         # Position and speed at time t
#         x_t_pos = x_t_reshaped[:, :, :2]
#         s_true  = x_t_reshaped[:, :, 2]

#         # Position at t+1 (ground truth)
#         next_pos = y_target_reshaped[:, :, :2]

#         # 4. Unpack y_pred (predicted acceleration, dir_sin, dir_cos)
#         y_pred_reshaped = tf.reshape(y_pred, (-1, 23, 3))  # (batch, 23, 3)
#         a_pred = y_pred_reshaped[:, :, 0]
#         dir_sin_pred = y_pred_reshaped[:, :, 1]
#         dir_cos_pred = y_pred_reshaped[:, :, 2]

#         # 5. Reconstruct
#         pred = tf.stack([a_pred, dir_sin_pred, dir_cos_pred], axis=-1)

#         # 6. Call physics-informed loss
#         return physics_informed_loss_accel(next_pos, pred, x_t_pos, s_true, delta_t=self.delta_t)
    

In [157]:
class PhysicsLossWrapper(tf.keras.losses.Loss):
    def __init__(self, delta_t=0.1):
        super().__init__()
        self.delta_t = delta_t

    def call(self, y_true, y_pred):
        # y_true: (batch, 276)
        # y_pred: (batch, 69)

        # Split x_t and y_target
        x_t_flat     = y_true[:, :138]
        y_target_flat = y_true[:, 138:]

        x_t = tf.reshape(x_t_flat, (-1, 23, 6))
        y_target = tf.reshape(y_target_flat, (-1, 23, 6))

        x_t_pos = x_t[:, :, :2]   # (x, y) at time t
        s_true  = x_t[:, :, 2]    # speed at time t
        next_pos = y_target[:, :, :2]  # true (x, y) at t+1

        y_pred = tf.reshape(y_pred, (-1, 23, 3))
        return physics_informed_loss_accel(next_pos, y_pred, x_t_pos, s_true, self.delta_t)


In [ ]:
# ╔════════════════════╗
# ║ 7. Compile & train ║
# ╚════════════════════╝

# ── 1)  Make sure we have a place to put checkpoints ─────────────────
WEIGHT_DIR = Path("../weights_physics")
WEIGHT_DIR.mkdir(parents=True, exist_ok=True)

ckpt_cb = keras.callbacks.ModelCheckpoint(
    filepath=(WEIGHT_DIR /
              "epoch_{epoch:03d}.weights.h5").as_posix(),
    monitor="val_loss",
    save_best_only=False,      # save every epoch → “periodic” archive
    save_weights_only=True,    # just the weights, not optimizer state
    verbose=0,
)

# ── 2)  Early-stopping ───────────────────────────────────────────────
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
    verbose=1,
)

# ── 3)  Compile the model ────────────────────────────────────────────
LR = 1e-4
model.compile(
    optimizer=keras.optimizers.Adam(LR),
    loss=PhysicsLossWrapper(delta_t=0.1),
    run_eagerly=True
)

# ── 4)  Fit – stop early, save weights each epoch ────────────────────
EPOCHS = 5   # high ceiling; early-stop decides real count
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stop, ckpt_cb],
    verbose=1,
)


Epoch 1/5
   3261/Unknown 8323s 3s/step - loss: 0.1171

/opt/anaconda3/envs/NFL_env/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3261/3261 ━━━━━━━━━━━━━━━━━━━━ 8908s 3s/step - loss: 0.1171 - val_loss: 0.0182
Epoch 2/5
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 8807s 3s/step - loss: 0.0182 - val_loss: 0.0139
Epoch 3/5
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 8782s 3s/step - loss: 0.0147 - val_loss: 0.0127
Epoch 4/5
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.0134

2025-04-30 10:01:01.378477: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


3261/3261 ━━━━━━━━━━━━━━━━━━━━ 8818s 3s/step - loss: 0.0134 - val_loss: 0.0120
Epoch 5/5
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 9376s 3s/step - loss: 0.0125 - val_loss: 0.0116
Restoring model weights from the end of the best epoch: 5.


In [159]:
# ╔═══════════════════╗
# ║ 8. Test Evaluation║
# ╚═══════════════════╝
test_loss = model.evaluate(test_ds, verbose=1)
print(f"\n✅  Test Physics-Informed Loss: {test_loss:.5f}")


593/593 ━━━━━━━━━━━━━━━━━━━━ 528s 889ms/step - loss: 0.0148

✅  Test Physics-Informed Loss: 0.01504


In [164]:
# ╔═══════════════╗
# ║ 9. Evaluation ║
# ╚═══════════════╝
# Simple end‑to‑end evaluation on a held‑out batch
loss_fn = PhysicsLossWrapper(delta_t=0.1)
X_batch, y_flat = next(iter(val_ds))

# Extract x_t (last frame's x, y coordinates) from input sequence
last_frame = X_batch[:, -1, :]
last_frame_reshaped = tf.reshape(last_frame, (-1, 23, 6))
x_t = last_frame_reshaped[:, :, :2]

y_pred = model(X_batch)
loss = loss_fn(y_flat, y_pred)

print("Validation Physics Loss (batch):", loss.numpy())


Validation Physics Loss (batch): 0.021965306
